In [1]:
import os
from datetime import datetime

PROJECT_ROOT_DIR = "/Users/gopora/MyStuff/Dev/Workspaces/Sandbox/TroubledLife"
DATASETS_DIR = os.path.join(PROJECT_ROOT_DIR, "data")
TF_LOG_DIR = os.path.join(PROJECT_ROOT_DIR, "tf_logs")
MODEL_CHECKPOINTS_DIR = os.path.join(PROJECT_ROOT_DIR, "model_checkpoints")
TRAINING_SET_DATA_FILE = "troubled_life_policy_train_data.csv"
TEST_SET_DATA_FILE = "troubled_life_policy_test_data.csv"

now = datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "{}/run-{}/".format(TF_LOG_DIR, now)


import data_preparation as dp

#dp.generate_troubled_life_policy_data(no_of_policies=10000, runtime=5, file_path=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))

#dp.generate_troubled_life_policy_data(no_of_policies=2000, runtime=5, file_path=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))

policy_histories_train = \
    dp.load_troubled_life_policy_data(file_path=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))

policy_histories_test = \
    dp.load_troubled_life_policy_data(file_path=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))

policy_histories_length_train, max_policy_history_length_train = \
    dp.get_policy_history_lengths(policy_histories=policy_histories_train)

policy_histories_length_test, max_policy_history_length_test = \
    dp.get_policy_history_lengths(policy_histories=policy_histories_test)

max_policy_history_length = max(max_policy_history_length_train, max_policy_history_length_test)



# Pad the histories up to maximum length of both, train and test set

# policy_histories_train = \
#     dp.pad_troubled_life_policy_histories(policy_histories=policy_histories_train,
#                                           policy_histories_lengths=policy_histories_length_train,
#                                           max_policy_history_length=max_policy_history_length)
# 
# policy_histories_test = \
#     dp.pad_troubled_life_policy_histories(policy_histories=policy_histories_test,
#                                           policy_histories_lengths=policy_histories_length_test,
#                                           max_policy_history_length=max_policy_history_length)
# 
# # # Save padded data, since always generating and padding takes too long
# policy_histories_train.to_csv(path_or_buf=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))
# policy_histories_test.to_csv(path_or_buf=os.path.join(DATASETS_DIR, TEST_SET_DATA_FILE))

# Extract features and labels from dataset as numpy.ndarray(s)
binary_classification = True

train_labels, train_features, train_seq_lengths =\
    dp.prepare_labels_features_lengths(policy_histories=policy_histories_train,
                                       policy_histories_lengths=policy_histories_length_train, 
                                       max_policy_history_length=max_policy_history_length,
                                       binary_classification=binary_classification)
test_labels, test_features, test_seq_lengths =\
    dp.prepare_labels_features_lengths(policy_histories=policy_histories_test,
                                       policy_histories_lengths=policy_histories_length_test, 
                                       max_policy_history_length=max_policy_history_length,
                                       binary_classification=binary_classification)

train_data = dp.TrainDataSet(train_labels=train_labels, train_features=train_features, train_seq_lengths=train_seq_lengths)

In [5]:
import tensorflow as tf
import gan as gan


g_learning_rate = 0.0001
d_real_learning_rate = 0.0001
d_fake_learning_rate = 0.0001
noise_dimensions = 10
batch_size = 200
g_n_outputs = 2
d_n_inputs = 2
d_n_outputs = 2 if binary_classification else max_policy_history_length

tf.reset_default_graph()
tf.set_random_seed(42)

noise = tf.placeholder(tf.float32, [batch_size, max_policy_history_length, noise_dimensions], name='noise')
noise_seq_length = tf.placeholder(tf.int32, [None], name='noise_seq_length')

X = tf.placeholder(tf.float32, [None, max_policy_history_length, d_n_inputs], name="X")
y = tf.placeholder(tf.int32, [None], name="y")
X_seq_length = tf.placeholder(tf.int32, [None], name="X_seq_length")

g_data = gan.generator(noise=noise, batch_size=batch_size, n_outputs=g_n_outputs, 
                       max_seq_length=max_policy_history_length, seq_length=noise_seq_length)

# Let's make sure the initial value at [:, 0, 1] is 0 (current capital at policy inception)
flat = tf.reshape(tensor=g_data, shape=[batch_size * max_policy_history_length * g_n_outputs])
flat2 = tf.Variable(initial_value=tf.zeros_like(flat), trainable=False, name="xyFlat")
flat3 = tf.assign(ref=flat2, value=flat)
flat4 = tf.scatter_update(flat3, [1 + i * max_policy_history_length * g_n_outputs for i in range(batch_size)],
                                 [0 for j in range(batch_size)])
g_ms_data = tf.reshape(tensor=flat4, shape=[batch_size, max_policy_history_length, g_n_outputs])

logits_d_for_X, y_pred_X = gan.discriminator(X=X, seq_length=X_seq_length, n_outputs=d_n_outputs)
logits_d_for_g, y_pred_g = gan.discriminator(X=g_ms_data, seq_length=noise_seq_length, n_outputs=d_n_outputs, reuse=True)

g_loss, accuracy_g = gan.generator_loss(batch_size=batch_size, logits_d=logits_d_for_g, logits_g = g_data)
d_loss_real, accuracy_d_real = gan.discriminator_loss_real(logits=logits_d_for_X, y=y)
d_loss_fake, accuracy_d_fake = gan.discriminator_loss_fake(batch_size=batch_size, logits=logits_d_for_g)

g_trainer = gan.generator_trainer(g_learning_rate, g_loss)
d_trainer_real = gan.discriminator_trainer_real(d_real_learning_rate, d_loss_real)
d_trainer_fake = gan.discriminator_trainer_fake(d_fake_learning_rate, d_loss_fake)

tf.summary.scalar('Generator_loss', g_loss)
tf.summary.scalar('Discriminator_loss_real', d_loss_real)
tf.summary.scalar('Discriminator_loss_fake', d_loss_fake)

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

In [6]:
import numpy as np

np.set_printoptions(formatter={'float_kind': (lambda x: "%.2f" % x)})

sess = tf.Session()

sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

saver = tf.train.Saver()

# n_epochs = 100
# 
# # Pre-train discriminator
# for epoch in range(n_epochs):
#     for batch in range(train_data.num_examples // batch_size):
#         noise_batch = np.random.normal(0, 1, size=[batch_size, max_policy_history_length, noise_dimensions])
#         noise_seq_length_batch = np.full(batch_size, max_policy_history_length)
# 
#         y_batch, X_batch, X_seq_length_batch = train_data.next_batch(batch_size)
# 
#         _, __, dLossReal, dLossFake, acc_real_train, acc_fake_train = \
#             sess.run([d_trainer_real, d_trainer_fake, d_loss_real, d_loss_fake, accuracy_d_real, accuracy_d_fake],
#                      {X: X_batch, y: y_batch, X_seq_length: X_seq_length_batch,
#                       noise: noise_batch, noise_seq_length: noise_seq_length_batch})
# 
#     print("Epoch:", epoch, "dLossReal:", dLossReal, "acc_real_train:", acc_real_train,
#           "dLossFake:", dLossFake, "acc_fake_train:", acc_fake_train)
# 
# saver.save(sess, os.path.join(MODEL_CHECKPOINTS_DIR, "discriminator.ckpt"))

In [7]:
#saver.restore(sess, os.path.join(MODEL_CHECKPOINTS_DIR, "discriminator.ckpt"))

#Train generator and discriminator together
n_epochs = 200

for epoch in range(n_epochs):
    for batch in range(train_data.num_examples // batch_size):
        noise_batch = np.random.normal(0, 1, size=[batch_size, max_policy_history_length, noise_dimensions])
        noise_seq_length_batch = np.full(batch_size, max_policy_history_length)
        
        # y_batch, X_batch, X_seq_length_batch = train_data.next_batch(batch_size)
        # 
        # # Train discriminator on both real and fake data
        # _, __, dLossReal, dLossFake, acc_real_train, acc_fake_train = \
        #     sess.run([d_trainer_real, d_trainer_fake, d_loss_real, d_loss_fake, accuracy_d_real, accuracy_d_fake],
        #              {X: X_batch, y: y_batch, X_seq_length: X_seq_length_batch,
        #               noise: noise_batch, noise_seq_length: noise_seq_length_batch})
        # 
        # # Train generator
        # noise_batch = np.random.normal(0, 1, size=[batch_size, max_policy_history_length, noise_dimensions])
    
        _, dLossFake, acc_d_fake, gLoss, acc_g, gData = sess.run([g_trainer, d_loss_fake, accuracy_d_fake,
                                                                         g_loss, accuracy_g, g_ms_data],
                                          feed_dict={noise: noise_batch, noise_seq_length: noise_seq_length_batch})

    print("Epoch:", epoch, 
          "dLossFake:", dLossFake, "acc_fake_train:", acc_d_fake,
          "gLoss:", gLoss, "acc_g:", acc_g)

    print(gData[0])


Epoch: 0 dLossFake: 0.6986 acc_fake_train: 0.225 gLoss: 2497.9 acc_g: 0.775
[[0.76 0.00]
 [0.94 -0.06]
 [1.12 0.04]
 [1.69 0.19]
 [1.33 0.12]
 [1.26 0.16]
 [1.37 0.17]
 [1.55 0.13]
 [1.71 0.21]
 [1.38 0.09]
 [1.48 0.09]
 [1.43 0.17]
 [1.74 0.16]
 [1.44 -0.01]
 [1.75 0.03]]


Epoch: 1 dLossFake: 0.718764 acc_fake_train: 0.165 gLoss: 2489.18 acc_g: 0.835
[[4.60 0.00]
 [5.85 0.10]
 [6.34 0.08]
 [5.87 0.22]
 [6.37 0.17]
 [6.23 0.40]
 [7.86 0.47]
 [7.76 0.31]
 [8.73 0.23]
 [8.16 0.58]
 [7.36 0.57]
 [5.82 0.79]
 [6.78 0.49]
 [6.84 0.61]
 [6.60 0.62]]


Epoch: 2 dLossFake: 0.794291 acc_fake_train: 0.12 gLoss: 2463.0 acc_g: 0.88
[[21.53 0.00]
 [21.03 0.57]
 [19.98 0.89]
 [30.12 1.05]
 [26.66 0.97]
 [27.09 1.20]
 [28.31 1.96]
 [24.69 1.57]
 [23.37 1.89]
 [27.33 1.92]
 [23.37 1.87]
 [26.18 1.58]
 [28.87 1.74]
 [27.53 1.52]
 [24.29 1.74]]


Epoch: 3 dLossFake: 1.09009 acc_fake_train: 0.025 gLoss: 2397.72 acc_g: 0.975
[[46.55 0.00]
 [57.57 0.86]
 [43.07 1.58]
 [51.68 2.28]
 [69.36 1.81]
 [76.89 2.03]
 [76.25 2.91]
 [68.07 3.35]
 [61.74 2.85]
 [65.78 3.31]
 [69.38 2.89]
 [58.11 2.20]
 [62.36 2.98]
 [68.55 2.74]
 [72.45 2.35]]


Epoch: 4 dLossFake: 1.81224 acc_fake_train: 0.025 gLoss: 2282.75 acc_g: 0.975
[[112.60 0.00]
 [110.27 1.30]
 [127.84 3.17]
 [137.65 4.34]
 [144.23 4.01]
 [118.78 4.22]
 [156.17 5.57]
 [126.01 3.38]
 [171.97 4.83]
 [157.69 4.01]
 [135.15 4.54]
 [139.03 4.97]
 [157.33 4.42]
 [147.90 3.75]
 [127.33 4.07]]


...

Epoch: 32 dLossFake: 18.3422 acc_fake_train: 0.0 gLoss: 265.211 acc_g: 1.0
[[1353.77 0.00]
 [1228.96 4.17]
 [1384.95 16.28]
 [1512.04 33.23]
 [1567.51 21.10]
 [1514.34 21.61]
 [1705.77 28.05]
 [1574.60 21.98]
 [1564.60 25.01]
 [1736.37 25.46]
 [1478.50 16.38]
 [1383.99 20.96]
 [1522.61 21.93]
 [1418.83 13.30]
 [1512.18 17.37]]


Epoch: 33 dLossFake: 19.4057 acc_fake_train: 0.0 gLoss: 267.531 acc_g: 1.0
[[1009.99 0.00]
 [1183.30 5.52]
 [1407.69 10.35]
 [1424.60 27.66]
 [1428.76 24.43]
 [1599.26 24.48]
 [1605.97 23.51]
 [1590.64 24.56]
 [1670.42 25.12]
 [1518.71 21.59]
 [1604.23 23.84]
 [1656.10 21.56]
 [1589.95 19.71]
 [1507.54 20.25]
 [1483.45 19.36]]


Epoch: 34 dLossFake: 19.4135 acc_fake_train: 0.0 gLoss: 229.548 acc_g: 1.0
[[1252.32 0.00]
 [1085.87 3.04]
 [1313.72 15.11]
 [1452.32 31.02]
 [1509.38 19.94]
 [1500.95 22.06]
 [1738.92 29.02]
 [1776.42 22.42]
 [1550.63 23.10]
 [1709.83 27.75]
 [1602.38 20.12]
 [1509.41 19.79]
 [1656.19 23.40]
 [1563.05 16.75]
 [1655.43 19.39]]


Epoch: 35 dLossFake: 19.864 acc_fake_train: 0.0 gLoss: 248.49 acc_g: 1.0
[[1408.63 0.00]
 [1341.46 4.87]
 [1395.63 17.17]
 [1504.38 35.11]
 [1426.35 21.52]
 [1652.81 23.72]
 [1566.66 23.59]
 [1581.29 24.79]
 [1531.36 23.52]
 [1385.43 21.21]
 [1551.66 21.44]
 [1581.84 18.60]
 [1603.59 18.27]
 [1595.84 20.87]
 [1695.19 21.77]]


KeyboardInterrupt: 